# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on the Ames Housing dataset!

## Objectives
You will be able to:
* Determine if it is necessary to perform normalization/standardization for a specific model or set of data
* Use standardization/normalization on features of a dataset
* Identify if it is necessary to perform log transformations on a set of features
* Perform log transformations on different features of a dataset
* Use statsmodels to fit a multiple linear regression model
* Evaluate a linear regression model by using statistical performance metrics pertaining to overall model and specific parameters


## The Ames Housing Data

Using the specified continuous and categorical features, preprocess your data to prepare for modeling:
* Split off and one hot encode the categorical features of interest
* Log and scale the selected continuous features

In [1]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']


## Continuous Features

In [3]:
# Log transform and normalize
ames_cont = ames[continuous]
ames_cont.head()

,LotArea,1stFlrSF,GrLivArea,SalePrice
0,8450,856,1710,208500
1,9600,1262,1262,181500
2,11250,920,1786,223500
3,9550,961,1717,140000
4,14260,1145,2198,250000


In [9]:
ames_cont.columns

Index(['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice'], dtype='object')

In [8]:
log_names = [f'{column}_log' for column in ames_cont.columns]
log_names

['LotArea_log', '1stFlrSF_log', 'GrLivArea_log', 'SalePrice_log']

In [11]:
ames_log = np.log(ames_cont)
ames_log.head()

,LotArea,1stFlrSF,GrLivArea,SalePrice
0,9.041922,6.752270,7.444249,12.247694
1,9.169518,7.140453,7.140453,12.109011
2,9.328123,6.824374,7.487734,12.317167
3,9.164296,6.867974,7.448334,11.849398
4,9.565214,7.043160,7.695303,12.429216


In [14]:
ames_log.columns = log_names # here you're adding that '_log' suffix on all the column names

Index(['LotArea_log', '1stFlrSF_log', 'GrLivArea_log', 'SalePrice_log'], dtype='object')

In [21]:
# normalize (substract mean and divide by std)
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)
ames_log_norm.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log
0,-0.133185,-0.803295,0.529078,0.559876
1,0.113403,0.418442,-0.381715,0.212692
2,0.419917,-0.576363,0.659449,0.733795
3,0.103311,-0.439137,0.541326,-0.437232
4,0.878108,0.112229,1.281751,1.014303


## Categorical Features

**One Hot Encoding** (Geeks for Geeks)

It refers to splitting the column which contains numerical categorical data to many columns depending on the number of categories present in that column. Each column contains “0” or “1” corresponding to which column it has been placed.

https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/

In [24]:
# One hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)
ames_ohe.head()

,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,SaleType_CWD,SaleType_Con,SaleType_ConLD,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## Combine Categorical and Continuous Features

In [25]:
# combine features into a single dataframe called preprocessed
preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)
preprocessed.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0.559876,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0.212692,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0.733795,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,-0.437232,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,1.014303,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


## Run a linear model with SalePrice as the target variable in statsmodels

In [31]:
preprocessed['SalePrice_log']

0       0.559876
1       0.212692
2       0.733795
3      -0.437232
4       1.014303
          ...   
1455    0.121392
1456    0.577822
1457    1.174306
1458   -0.399519
1459   -0.306589
Name: SalePrice_log, Length: 1460, dtype: float64

In [34]:
# Your code here
X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

import statsmodels.api as sm
X_int = sm.add_constant(X.values)
model = sm.OLS(y,X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     156.5
Date:                Sun, 04 Oct 2020   Prob (F-statistic):               0.00
Time:                        13:54:43   Log-Likelihood:                -738.14
No. Observations:                1460   AIC:                             1572.
Df Residuals:                    1412   BIC:                             1826.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1317      0.263     -0.500      0.617      -0.648       0.385
x1             0.1033      0.019      5.475      0.000       0.066       0.140
x2             0.1371      0.016      8.584      0.000       0.106       0.168
x3             0.3768      0.016     24.114      0.000       0.346       0.407
x4            -0.1715      0.079     -2.173      0.030      -0.326      -0.017
x5            -0.4203      0.062     -6.813      0.000      -0.541      -0.299
x6            -0.1403      0.093     -1.513      0.130      -0.322       0.042
x7            -0.0512      0.060     -0.858      0.391      -0.168       0.066
x8            -0.9999      0.088    -11.315      0.000      -1.173      -0.827
x9            -0.3820      0.050     -7.613      0.000      -0.480      -0.284
x10           -0.6692      0.055    -12.111      0.000      -0.778      -0.561
x11            0.2285      0.215      1.061      0.289      -0.194       0.651
x12            0.5861      0.304      1.927      0.054      -0.010       1.183
x13            0.3151      0.155      2.029      0.043       0.010       0.620
x14            0.0331      0.195      0.169      0.865      -0.350       0.416
x15            0.0161      0.196      0.082      0.935      -0.368       0.400
x16            0.2999      0.079      3.803      0.000       0.145       0.455
x17            0.1178      0.245      0.480      0.631      -0.364       0.599
x18            0.1748      0.065      2.676      0.008       0.047       0.303
x19            1.0666      0.193      5.526      0.000       0.688       1.445
x20            0.8768      0.194      4.512      0.000       0.496       1.258
x21            0.9961      0.162      6.151      0.000       0.678       1.314
x22            1.1023      0.152      7.264      0.000       0.805       1.400
x23           -0.2131      0.180     -1.182      0.237      -0.567       0.141
x24            0.0529      0.318      0.167      0.868      -0.571       0.677
x25           -0.4627      0.171     -2.711      0.007      -0.798      -0.128
x26           -0.6498      0.137     -4.735      0.000      -0.919      -0.381
x27           -0.2102      0.144     -1.456      0.146      -0.493       0.073
x28           -0.0761      0.119     -0.641      0.522      -0.309       0.157
x29           -0.0823      0.129     -0.638      0.523      -0.335       0.171
x30           -0.7613      0.124     -6.143      0.000      -1.004      -0.518
x31           -0.0980      0.126     -0.777      0.437      -0.346       0.150
x32           -0.9618      0.160     -6.014      0.000      -1.276      -0.648
x33           -0.6918      0.159     -4.351      0.000      -1.004      -0.380
x34           -0.2553      0.131     -1.944      0.052      -0.513       0.002
x35           -0.4407      0.120     -3.664      0.000      -0.677      -0.205
x3

## Run the same model in scikit-learn

In [37]:
# Your code here - Check that the coefficients and intercept are the same as those from Statsmodels
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [38]:
# coefficients
linreg.coef_

array([ 0.10327192,  0.1371289 ,  0.37682133, -0.1714623 , -0.42033885,
       -0.14034113, -0.05120194, -0.99986001, -0.38202198, -0.66924909,
        0.22847737,  0.5860786 ,  0.31510567,  0.0330941 ,  0.01608664,
        0.29985338,  0.11784232,  0.17480326,  1.06663561,  0.87681007,
        0.99609131,  1.10228499, -0.21311107,  0.05293276, -0.46271253,
       -0.64982261, -0.21019239, -0.07609253, -0.08233633, -0.76126683,
       -0.09799942, -0.96183328, -0.69182575, -0.2553217 , -0.44067351,
       -0.01595046, -0.26762962,  0.36313165,  0.36259667, -0.93504972,
       -0.69976325, -0.47543141, -0.23309732,  0.09502969,  0.42957077,
        0.0056924 ,  0.12762613])

In [39]:
# intercept
linreg.intercept_

# yes, the answer is the same as coef/const in the model summary

-0.1316973691667042

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- LotArea: 14977
- 1stFlrSF: 1976
- GrLivArea: 1976
- BldgType: 1Fam
- KitchenQual: Gd
- SaleType: New
- MSZoning: RL
- Street: Pave
- Neighborhood: NridgHt

## Summary
Congratulations! You pre-processed the Ames Housing data using scaling and standardization. You also fitted your first multiple linear regression model on the Ames Housing data using statsmodels and scikit-learn!